In [ ]:
from selenium.webdriver.chrome.service import Service
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import undetected_chromedriver as uc

def wait_claude(driver):
    """
    Wait for Claude to complete response.
    Check by polling if the "stop response" button exists.
    This function waits until the "stop response" button disappears.
    """
    start_time = time.time()
    # Note: If your Claude interface is in another language, modify the aria-label text here
    stop_button_selector = 'button[aria-label="応答を停止"]'
    
    # Step 1: First, wait for "stop response" button to appear, confirming Claude has started generating
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, stop_button_selector))
        )
        # print("Claude has started generating...")
    except TimeoutException:
        print("Warning: Stop button did not appear. Claude might not be generating a response.")
        return

    # Step 2: Wait for "stop response" button to disappear, indicating generation is complete
    try:
        WebDriverWait(driver, 600).until_not(
            EC.presence_of_element_located((By.CSS_SELECTOR, stop_button_selector))
        )
        # print("Claude response finished.")
    except TimeoutException:
        raise TimeoutException("Waiting for Claude timed out after 10 minutes! The 'stop generating' button did not disappear.")


def prompting_claude(prompt, driver):
    """
    Send prompt to Claude and get response.
    Use data-test-render-count="1" to precisely locate the latest reply.
    """
    try:
        # 1. Locate input box and enter text
        input_selector = 'div[role="textbox"]'
        input_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, input_selector))
        )
        input_element.click()
        input_element.clear()
        
        # Simulate human typing
        for char in prompt:
            input_element.send_keys(char)
            time.sleep(random.uniform(0.01, 0.03))
        
        # 2. Find and click "send message" button
        send_button_selector = 'button[aria-label="メッセージを送信"]:not([disabled])'
        send_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, send_button_selector))
        )
        driver.execute_script("arguments[0].click();", send_button)
        
        # 3. Wait for Claude to complete generation
        wait_claude(driver)
        
        # 4. Precisely wait and locate the latest response container
        final_response_selector = 'div[data-test-render-count="1"] .standard-markdown'
        
        try:
            # Wait for this unique, latest response element to appear
            final_response_element = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, final_response_selector))
            )
            print("Successfully located the unique final response element.")
        except TimeoutException:
            print("Error: Timed out waiting for the final response container (data-test-render-count='1') to appear.")
            driver.save_screenshot('claude_final_response_timeout.png')
            
            # --- Fallback method ---
            print("Trying fallback method to get the last response...")
            all_responses = driver.find_elements(By.CSS_SELECTOR, 'div[data-is-response="true"]')
            if all_responses:
                print("Fallback successful. Found response via data-is-response.")
                return all_responses[-1].text
            return None

        # 5. Get and return text
        return final_response_element.text

    except Exception as e:
        print(f"An unexpected error occurred in prompting_claude: {e}")
        driver.save_screenshot('claude_unexpected_error.png')
        return None

def create_claude():
    """
    Create and initialize a Chrome browser instance for Claude
    """
    options = uc.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    chromedriver_path = r"./chromedriver.exe" # Change to your path
    
    driver = uc.Chrome(service=Service(chromedriver_path), options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    driver.get("https://claude.ai/new")
    
    input("Please log in to Claude.ai in the browser window, then press Enter here to continue...")
    
    time.sleep(random.uniform(2, 3))
    return driver

def prompt_claude(driver, mondai):
    ans = prompting_claude(mondai, driver) 
    return ans

def close_claude(driver):
    """
    Close browser
    """
    if driver:
        driver.close()
        driver.quit()


driver = None
try:
    driver = create_claude()
    
    my_prompt = 'Generate 20 random numbers between 10 and 50, output in Python list format.'
    
    print(f"Sending prompt to Claude: '{my_prompt}'")
    ans = prompt_claude(driver, my_prompt)
    
    if ans:
        print("\n--- Claude's Response ---")
        print(ans)
        print("-------------------------\n")
    else:
        print("Failed to get a response from Claude.")
        
except Exception as e:
    print(f"An error occurred in main: {e}")
finally:
    print("Closing the browser...")
    close_claude(driver)